## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error

## Prepare data for model training

In [2]:
with open("../input/workation-price-prediction-preprocess-data-v2/Workation_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

train_df = train_df[~((train_df['Per Person Price'] < 7.3) | (np.exp(train_df['Per Person Price']) > 150000))].copy()

del processed_data
gc.collect()

46

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'Per Person Price'].values
Ytrain = train_df['Per Person Price'].values
Ytrain_strat = pd.qcut(train_df['Per Person Price'].values, q=10, labels=range(0,10))
Xtest = test_df.values

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (20989, 5752) 
Ytrain: (20989,) 
Xtest: (9000, 5752)


0

## Build and validate the model

In [4]:
FOLD = 10
NUM_SEED = 3

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_xgb = np.zeros((Ytrain.shape[0], NUM_SEED))
y_pred_final_xgb = np.zeros((Xtest.shape[0], NUM_SEED))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain_strat)):
        counter += 1

        train_x, train_y = Xtrain[train], Ytrain[train]
        val_x, val_y = Xtrain[val], Ytrain[val]

        model = XGBRegressor(
            objective='reg:squarederror',
            eval_metric='rmse',
            booster='gbtree',
            sample_type='uniform',
            tree_method='gpu_hist',
            grow_policy='lossguide',
            num_round=8000,
            max_depth=12, 
            max_leaves=135,
            learning_rate=0.093,
            subsample=0.98,
            colsample_bytree=0.61,
            min_child_weight=11,
            reg_lambda=0.0001,
            verbosity=0
        )

        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (val_x, val_y)], 
                  early_stopping_rounds=200, verbose=50)

        y_pred = model.predict(val_x, ntree_limit=model.best_ntree_limit)
        y_pred_meta_xgb[val, sidx] += y_pred
        y_pred_final_xgb[:, sidx] += model.predict(Xtest, ntree_limit=model.best_ntree_limit)
        
        score = np.sqrt(mean_squared_log_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final_xgb = y_pred_final_xgb / float(FOLD)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

[0]	validation_0-rmse:8.41608	validation_1-rmse:8.41672
[50]	validation_0-rmse:0.13180	validation_1-rmse:0.18287
[99]	validation_0-rmse:0.09629	validation_1-rmse:0.16614
Seed-24 | Fold-0 | OOF Score: 0.015400947218408513
[0]	validation_0-rmse:8.41630	validation_1-rmse:8.41393
[50]	validation_0-rmse:0.13248	validation_1-rmse:0.16920
[99]	validation_0-rmse:0.09656	validation_1-rmse:0.15663
Seed-24 | Fold-1 | OOF Score: 0.014537529075796947
[0]	validation_0-rmse:8.41604	validation_1-rmse:8.41665
[50]	validation_0-rmse:0.13214	validation_1-rmse:0.17322
[99]	validation_0-rmse:0.09628	validation_1-rmse:0.15610
Seed-24 | Fold-2 | OOF Score: 0.014428599147461982
[0]	validation_0-rmse:8.41624	validation_1-rmse:8.41596
[50]	validation_0-rmse:0.13263	validation_1-rmse:0.17739
[99]	validation_0-rmse:0.09687	validation_1-rmse:0.16554
Seed-24 | Fold-3 | OOF Score: 0.015248453722584817
[0]	validation_0-rmse:8.41600	validation_1-rmse:8.41702
[50]	validation_0-rmse:0.13254	validation_1-rmse:0.17214
[99

In [5]:
np.savez_compressed('./XGB_Meta_Features.npz',
                    y_pred_meta_xgb=y_pred_meta_xgb, 
                    oof_score=oof_score,
                    y_pred_final_xgb=y_pred_final_xgb)

In [6]:
df = pd.DataFrame(y_pred_final_xgb)
df.head()

,0,1,2
0,9.842920,9.842694,9.840515
1,9.325997,9.316101,9.306221
2,8.654737,8.647831,8.647895
3,8.866700,8.863373,8.863447
4,9.985111,9.988617,9.981252
